In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import PyPDFLoader
from pydantic import BaseModel, Field
import instructor
from typing import List
import pandas as pd
from groq import Groq
from dotenv import load_dotenv
import json
import os

In [2]:
load_dotenv()
# Configure Groq API
groq_api_key = os.getenv("GROQ_API_KEY")

## Extract PDF text

In [3]:
#loader = PyPDFLoader("CV_avec_profil.pdf")
loader = PyPDFLoader("CV_sans_profil.pdf")
pages = loader.load_and_split()
text = " ".join(list(map(lambda page: page.page_content, pages)))
print(text)

CENTRES
D’ I NTÉRÊT
Thomas Garcia
Football, Vélo, Natation
123 Anywhere St., AnyCity,
hello@reallygreatsite.com
123-456-7890
CONTACT
Arabe       :  Bilingue
Anglais     :  Niveau scolaire
Espagnol  :  Niveau scolaire
COMPÉTENCES
FORMATI ON
EXPÉRI ENCES
LANGUES
Permis
Durance Média
Bio&Bon
12/12/2024
12/12/2024
CHARGÉE DE COMMUNICATION & MARKETING
Lancement complète d'une marque
Community manager : gestion des réseaux sociaux
RP  (communiqués et dossiers de presse)
Print (roll up; adhésifs, catalogue)
 ASSISTANTE MARKETING SERVICE SUPPORT
Salons / Forums
Organisation des visites lycées pour les chargés de recrutement
Fitissimo
Tempo
12/12/2024
12/12/2024
CHARGÉE DE COMMUNICATION
Réalisation kakémonos, banderoles, affiches, post sur les réseaux sociaux.
Mise en place d'un jeu concours : prospection des lots
Service accueil et photographie
CHARGÉ DE COMMUNICATION
Réalisation plaquette commerciale, logo, affiches ...
Organisation d'événements (cérémonie, tournoi de pétanque)
Gestion admini

## Extract WORD text

In [17]:
loader = Docx2txtLoader("cv_word.docx")
pages = loader.load_and_split()
text = " ".join(list(map(lambda page: page.page_content, pages)))
print(text)

Chanchal Sharma





Chef de bureau



(718) 555–0100

chanchals@example.com

4567 Main Street City, ST 98052

www.interestingsite.com





Expérience

Chef d’entreprise Opérateur téléphonique 

January 20XX - Actuel 

Résumez vos responsabilités et réalisations clés. Le cas échéant, utilisez la langue et les mots que vous trouvez dans la description de poste spécifique. Soyez concis et ciblez les zones clés de 3-5.

Gestionnaire Office, Publication de nœuds 

Mars 20XX – Décembre 20XX 

Résumez vos responsabilités et réalisations clés. Là encore, profitez de toutes les occasions pour utiliser les mots que vous trouvez dans la description du poste. Soyez bref.

Chef de bureau Location de vidéos 

Août 20XX – Mars 20XX 

Résumez vos responsabilités et réalisations clés. Là encore, profitez de toutes les occasions pour utiliser les mots que vous trouvez dans la description du poste. Soyez concis et ciblez les zones clés de 3-5.



Éducation



Sep 20XX - Mai 20XX

Degré d’association, H.

## Pydantic class to extract information

In [4]:
default = ""

class Formation(BaseModel):
    """Formations que le candidat a suivi"""
    dates: str = Field(default=default)
    intitule_formation: str = Field(default=default)
    ecole: str = Field(default=default)

class Experience(BaseModel):
    """Expériences professionnelles du candidat"""
    dates: str = Field(default=default)
    nom_entreprise: str = Field(default=default)
    intitule_poste: str = Field(default=default)
    missions: List[str] = Field(default=[])

class Competence(BaseModel):
    """Compétence du candidat"""
    nom_competence: str = Field(default=default)
    niveau: str = Field(default=default)

class Langue(BaseModel):
    langue: str = Field(default=default)
    niveau: str = Field(default=default)

class Hobby(BaseModel):
    type_hobby: str = Field(default=default)
    nom_hobby: str = Field(default=default)

class CvExtractor(BaseModel):
    """Informations à extraire du CV du candidat"""
    nom: str = Field(default=default)
    prenom: str = Field(default=default)
    email: str = Field(default=default)
    adresse: str = Field(default=default)
    linkedin: str = Field(default=default)
    formations: List[Formation]
    experiences: List[Experience]
    competences: List[Competence]
    langues: List[Langue]
    centres_interets: List[Hobby]

class Raisonnement(BaseModel):
    extraction_cv: CvExtractor
    reponse_finale: str = Field(description="profil du candidat écrit en français en fonction des informations extraites du CV")

## Prompt Gen AI API to retrieve extracted Informations

In [18]:
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

In [20]:
sys_prompt = """Tu es une IA experte dans l'analyse des CV de candidats. 

J'aimerais analyser le CV d'un candidat et savoir quelle est son profil.

Ta tâche sera d'abord d'extraire les informations du CV sous formes de sections en suivant la structure du JSON Schema Raisonnement puis d'en déduire le profil du candidat dans "reponse_finale".

Tu extrairas les informations du CV et n'inventeras pas d'informations, il est très important que tu suives la structure du Schéma Pydantic donné en paramètre !"""

input_prompt = f"""
Contenu du CV brut : {text}

Réponse :
"""

In [31]:
sys_prompt

'Tu es une IA experte dans l\'analyse des CV de candidats. \n\nJ\'aimerais analyser le CV d\'un candidat et savoir quelle est son profil.\n\nTa tâche sera d\'abord d\'extraire les informations du CV sous formes de sections en suivant la structure du JSON Schema Raisonnement puis d\'en déduire le profil du candidat dans "reponse_finale".\n\nTu extrairas les informations du CV et n\'inventeras pas d\'informations, il est très important que tu suives la structure du Schéma Pydantic donné en paramètre !'

In [45]:
def extract_cv(client, sys_prompt, input_prompt):
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role":"user", "content": input_prompt}
            ],
            #model="llama3-8b-8192",
            model="llama-3.3-70b-versatile",
            response_model=Raisonnement,
        )
        return response
    except Exception as e:
        print(f"Erreur dans l'analyse du CV : {e}")
        return None

In [46]:
response = extract_cv(client, sys_prompt, input_prompt)
if response:
    response = json.loads(response.model_dump_json())
    print(response)
else:
    print("Error parsing the resume.")

{'extraction_cv': {'nom': 'Chanchal', 'prenom': 'Sharma', 'email': 'chanchals@example.com', 'adresse': '4567 Main Street City, ST 98052', 'linkedin': 'www.interestingsite.com', 'formations': [{'dates': 'Sep 20XX - Mai 20XX', 'intitule_formation': 'Degré d’association, H.R. Gestion', 'ecole': 'Bellows College'}], 'experiences': [{'dates': 'January 20XX - Actuel', 'nom_entreprise': '', 'intitule_poste': 'Chef d’entreprise Opérateur téléphonique', 'missions': ['Gestion de projet', 'Analyse des données', 'Communication', 'Organisation', 'Résolution des problèmes', 'Gestion']}, {'dates': 'Mars 20XX – Décembre 20XX', 'nom_entreprise': '', 'intitule_poste': 'Gestionnaire Office, Publication de nœuds', 'missions': []}, {'dates': 'Août 20XX – Mars 20XX', 'nom_entreprise': '', 'intitule_poste': 'Chef de bureau Location de vidéos', 'missions': []}], 'competences': [{'nom_competence': 'Gestion de projet', 'niveau': ''}, {'nom_competence': 'Analyse des données', 'niveau': ''}, {'nom_competence': 'C

In [47]:
df_formations = pd.DataFrame(response["extraction_cv"]["formations"])
df_formations = df_formations.rename(columns={"dates": "Dates", "intitule_formation": "Formation", "ecole": "Ecole"})

df_experiences = pd.DataFrame(response["extraction_cv"]["experiences"])
df_experiences = df_experiences.rename(columns={"dates": "Dates", "nom_entreprise": "Entreprise", "intitule_poste":"Poste", "missions": "Missions"})

df_competences = pd.DataFrame(response["extraction_cv"]["competences"])
df_competences = df_competences.rename(columns={"nom_competence":"Competence","niveau":"Niveau"})

df_langues = pd.DataFrame(response["extraction_cv"]["langues"])
df_langues = df_langues.rename(columns={"langue":"Langue", "niveau":"Niveau"})

df_centres_interets = pd.DataFrame(response["extraction_cv"]["centres_interets"])
df_centres_interets = df_centres_interets.rename(columns={"type_hobby":"Catégorie", "nom_hobby":"Hobby"})

del response["extraction_cv"]["formations"]
del response["extraction_cv"]["experiences"]
del response["extraction_cv"]["competences"]
del response["extraction_cv"]["centres_interets"]
del response["extraction_cv"]["langues"]

df_informations = pd.DataFrame(response["extraction_cv"], index=[0])
df_informations = df_informations.rename(columns={"nom":"Nom","prenom":"Prenom","email":"Email","adresse":"Adresse","linkedin":"Profil linkedin"})

In [48]:
response["reponse_finale"]

'Chanchal Sharma est un chef de bureau expérimenté avec des compétences en gestion de projet, analyse des données, communication, organisation, résolution des problèmes et gestion. Il a occupé plusieurs postes, notamment celui de chef d’entreprise opérateur téléphonique, gestionnaire office et chef de bureau. Il détient un diplôme d’associé en gestion des ressources humaines de Bellows College.'

In [49]:
df_informations

,Nom,Prenom,Email,Adresse,Profil linkedin
0,Chanchal,Sharma,chanchals@example.com,"4567 Main Street City, ST 98052",www.interestingsite.com


In [50]:
df_formations

,Dates,Formation,Ecole
0,Sep 20XX - Mai 20XX,"Degré d’association, H.R. Gestion",Bellows College


In [51]:
df_experiences

,Dates,Entreprise,Poste,Missions
0,January 20XX - Actuel,,Chef d’entreprise Opérateur téléphonique,"[Gestion de projet, Analyse des données, Commu..."
1,Mars 20XX – Décembre 20XX,,"Gestionnaire Office, Publication de nœuds",[]
2,Août 20XX – Mars 20XX,,Chef de bureau Location de vidéos,[]


In [52]:
df_competences

,Competence,Niveau
0,Gestion de projet,
1,Analyse des données,
2,Communication,
3,Organisation,
4,Résolution des problèmes,
5,Gestion,


In [53]:
df_langues

""


In [54]:
df_centres_interets

""
